# Create a pretrained RESNET34

Where to find pretrained model ?

RUN wget -O /root/.torch/models/vgg16_bn-6c64b313.pth https://download.pytorch.org/models/vgg16_bn-6c64b313.pth

RUN wget -O /root/.torch/models/resnet34-333f7ec4.pth https://download.pytorch.org/models/resnet34-333f7ec4.pth

RUN wget -O /data/models/ColorizeArtistic_gen.pth https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0 

In [9]:
import fastai
from fastai.vision.all import *
import pathlib
from pathlib import Path

from fastai.vision.core import *
from fastai.vision.data import *

In [3]:
# import torch
# #https://pytorch.org/hub/pytorch_vision_resnet/

# model_resnet34 = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)


## Prepare data

In [10]:
# Set those variables if running independently from the data set download
dest='../storage/pix2pix/data'
path=Path(dest,'maps')
pathXs=path/'Xs'
pathYs=path/'Ys'

iil = ImageImageList.from_folder(pathXs, presort=True).split_by_folder(train='train', valid='valid')

def getYsfromXs(x:Path, xFolder:str='Xs', yFolder:str='Ys'):
    index=x.parts.index(xFolder)
    y=Path(*x.parts[:index]).joinpath(Path(yFolder,*x.parts[index+1:]))
    return y

def get_data(iil:ImageImageList, bs:int,size:int):
    ''' Takes an ImageImageList in iil and return a databunch with images of size=size and training batch size= bs '''
    data = (iil.label_from_func(getYsfromXs)
           .transform(get_transforms(max_zoom=2.), size=size, tfm_y=True)
           .databunch(bs=bs).normalize(imagenet_stats, do_y=True))

    data.c = 3
    return data

bs = 2
size = 128
data = get_data(iil=iil, bs=bs,size=size)
data.show_batch(ds_type=DatasetType.Valid, rows=2, figsize=(9,9))

NameError: name 'ImageImageList' is not defined

### Redo ImageImageList in fastai v2

In [ ]:
iib = DataBlock(blocks=(ImageBlock, ImageBlock), 
                 get_items=get_image_files(path=pathXs), 
                 splitter=GrandparentSplitter(train_name='train', valid_name='valid'),
                 get_y=getYsfromXs,
                 item_tfms=Resize(460),
                 batch_tfms=aug_transforms(size=224))

DataBlock(blocks=None, dl_type=None, getters=None, n_inp=None, item_tfms=None, batch_tfms=None, get_items=None, splitter=None, get_y=None, get_x=None)